# Limpiar Coordenadas para Flood

## Importar librerías

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.impute import SimpleImputer
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from time import sleep

In [35]:
geolocator = Nominatim(user_agent="MozillaFirefox", timeout=10)

## Cargar base de datos

03desastreslimpio.csv

In [36]:
df = pd.read_csv('../../Data/03Limpio/03desastreslimpio.csv')
df.head(10)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
0,1900-9002-CPV,1900,9002,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,...,NaN,1900,0,0,1900,0,0,11000,0,0
1,1900-9001-IND,1900,9001,Climatological,Drought,Drought,India,IND,Southern Asia,Asia,...,NaN,1900,0,0,1900,0,0,1250000,0,0
2,1902-0012-GTM,1902,12,Geophysical,Earthquake,Ground movement,Guatemala,GTM,Central America,Americas,...,-91,1902,4,18,1902,4,18,2000,0,843726
3,1902-0003-GTM,1902,3,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,4,8,1902,4,8,1000,0,0
4,1902-0010-GTM,1902,10,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,10,24,1902,10,24,6000,0,0
5,1903-0006-CAN,1903,6,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1903,4,29,1903,4,29,76,23,0
6,1903-0012-COM,1903,12,Geophysical,Volcanic activity,Ash fall,Comoros (the),COM,Eastern Africa,Africa,...,NaN,1903,0,0,1903,0,0,17,0,0
7,1904-0003-BGD,1904,3,Meteorological,Storm,Tropical cyclone,Bangladesh,BGD,Southern Asia,Asia,...,NaN,1904,11,0,1904,11,0,0,0,0
8,1905-0005-CAN,1905,5,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1905,8,13,1905,8,13,18,18,0
9,1905-0003-IND,1905,3,Geophysical,Earthquake,Ground movement,India,IND,Southern Asia,Asia,...,76.16,1905,4,4,1905,4,4,20000,0,812477


## EDA Inicial Básico

In [37]:
df.shape

(16636, 26)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Dis No             16636 non-null  object
 1   Year               16636 non-null  int64 
 2   Seq                16636 non-null  int64 
 3   Disaster Subgroup  16636 non-null  object
 4   Disaster Type      16636 non-null  object
 5   Disaster Subtype   13313 non-null  object
 6   Country            16636 non-null  object
 7   ISO                16636 non-null  object
 8   Region             16636 non-null  object
 9   Continent          16636 non-null  object
 10  Location           14825 non-null  object
 11  Origin             4085 non-null   object
 12  Associated Dis     3593 non-null   object
 13  Dis Mag Value      16636 non-null  int64 
 14  Dis Mag Scale      15416 non-null  object
 15  Latitude           2775 non-null   object
 16  Longitude          2775 non-null   objec

## Filtro Disaster Type == Flood

In [39]:
filter = df['Disaster Type'] == 'Flood'
df_flood = df[filter]

In [40]:
df_flood.head(2)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
12,1906-0023-BEL,1906,23,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,NaN,1906,5,14,1906,5,14,6,0,0
13,1906-0024-BEL,1906,24,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,NaN,1906,4,0,1906,4,0,0,0,0


In [41]:
df_flood.shape

(5808, 26)

In [42]:
df_flood['Disaster Type'].dtype

dtype('O')

In [67]:
df_flood['Latitude'].isnull().sum()

0

In [66]:
df_flood['Longitude'].isnull().sum()

0

# Limpieza de coordenadas

## Función para limpiar coordenadas filtrado para df_flood

In [43]:
# Initializar geolocator
geolocator = Nominatim(user_agent="eqlimpiarcoord")

# Convertir 'Latitude' y 'Longitude' a string
df_flood['Latitude'] = df_flood['Latitude'].astype(str)
df_flood['Longitude'] = df_flood['Longitude'].astype(str)

# Limpiar coordenadas de latitude y longitude
df_flood['Latitude'] = df_flood['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x))
df_flood['Longitude'] = df_flood['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x))

# Remover puntos finales
df_flood['Latitude'] = df_flood['Latitude'].str.rstrip('.')
df_flood['Longitude'] = df_flood['Longitude'].str.rstrip('.')

# Añadir valores anómalos de latitud y longitud a listas
anomalous_lat = []
anomalous_lon = []

# Función para convertir coordenadas y detectar anomalías
def convert_coordinates(x, convert_nan=True):
    if not x:
        return np.nan

    # Remover caracteres 'N' y 'E'
    x = x.replace(' N', '').replace(' E', '')

    # Remover puntos extra en decimales
    x = re.sub('^(\d+\.\d{2})\..*', r'\1', x)

    # Conversión a negativo para S y W
    try:
        value = float(x)
        if x[-1] == 'S' or x[-1] == 'W':
            return -value
        else:
            return value
    except ValueError:
        return np.nan

# Convertir a float
df_flood['Latitude'] = df_flood['Latitude'].apply(convert_coordinates)
df_flood['Longitude'] = df_flood['Longitude'].apply(convert_coordinates)

# Redondear decimales
df_flood['Latitude'] = df_flood['Latitude'].round(2)
df_flood['Longitude'] = df_flood['Longitude'].round(2)

# Identificar valores anómalos de latitude y longitude
for index, row in df_flood.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']

    if latitude < -90 or latitude > 90:
        anomalous_lat.append(index)

    if longitude < -180 or longitude > 180:
        anomalous_lon.append(index)

# Revisar inconsistencias de cantidad entre anómalos de latitude y longitude
if len(anomalous_lat) != len(anomalous_lon):
    print("Valores inconsistentes de latitud y longitud.")
    # Gestionar la inconsistencia, como remover los valores extras o ajustar las listas
    # Por ejemplo, remover los valores extra de latitud:
    anomalous_lat = anomalous_lat[:len(anomalous_lon)]
    print("Longitud de latitud anómala ajustada:", len(anomalous_lat))

# Crear nuevo DataFrame con coordenadas limpias y emparejadas
df_eqcleaned = df_flood.copy()

# Función para rellenar coordenadas anómalas usando geocoding
def fill_anomalous_coordinates(row):
    if row.name in anomalous_lat and row.name in anomalous_lon:
        try:
            location = geolocator.reverse((row['Latitude'], row['Longitude']), timeout=10)
            if location and location.latitude is not None and location.longitude is not None:
                row['Latitude'] = location.latitude
                row['Longitude'] = location.longitude
                row['Location'] = location.address
                row['Country'] = location.raw['address'].get('country')
        except:
            pass
    return row

# Rellenar coordenadas anómalas
df_eqcleaned = df_eqcleaned.apply(fill_anomalous_coordinates, axis=1)

# Borrar filas con location y country no emparejadas
df_eqcleaned = df_eqcleaned[~((df_eqcleaned['Latitude'].isin(anomalous_lat)) & (df_eqcleaned['Longitude'].isin(anomalous_lon)))]

# Borrar filas con valores null o nan en columnas de Latitud y Longitud
df_eqcleaned = df_eqcleaned.dropna(subset=['Latitude', 'Longitude'])

# Convertir Latitud y Longitud a float con 2 decimales
df_eqcleaned['Latitude'] = df_eqcleaned['Latitude'].astype(float).round(2)
df_eqcleaned['Longitude'] = df_eqcleaned['Longitude'].astype(float).round(2)

# # Exportar el DataFrame limpio a archivo
# df_eqcleaned.to_csv('floodfilteredcleanedgeoloc.csv', index=False)


C:\Users\AleEng\AppData\Local\Temp\ipykernel_5580\3200028670.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flood['Latitude'] = df_flood['Latitude'].astype(str)
C:\Users\AleEng\AppData\Local\Temp\ipykernel_5580\3200028670.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flood['Longitude'] = df_flood['Longitude'].astype(str)
C:\Users\AleEng\AppData\Local\Temp\ipykernel_5580\3200028670.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Valores inconsistentes de latitud y longitud.
Longitud de latitud anómala ajustada: 0


1. Definimos la función fill_anomalous_latlon que toma una fila como entrada. Si los valores de latitud y longitud de la fila se encuentran en las listas anomalous_lat y anomalous_lon, respectivamente, la función intenta geocodificar la ubicación mediante la columna Location o la columna Country. Si se encuentra una ubicación válida, se actualizan los valores de latitud y longitud de la fila.

2. La llamada a la función sleep(1) agrega un retraso de 1 segundo entre las solicitudes de geolocalización para cumplir con la política de uso del servicio de geocodificación.

3. Luego creamos un objeto geolocalizador utilizando el geocodificador Nominatim de la biblioteca geopy.

4. Finalmente, aplicamos la función fill_anomalous_latlon a cada fila del DataFrame usando el método apply con axis=1, lo que indica que la función debe aplicarse por filas.

## Función con geopy para limpiar coordenadas

## Tests de verificación de limpieza de coordenadas

In [44]:
df_flood[['Latitude', 'Longitude', 'Location', 'Country']][700:800:]

,Latitude,Longitude,Location,Country
2695,NaN,NaN,Lake Malawi Shores,Malawi
2696,NaN,NaN,Northeast,Nicaragua
2700,NaN,NaN,NaN,Pakistan
2702,NaN,NaN,Cotabato,Philippines (the)
2710,NaN,NaN,Madeira Island,Portugal
...,...,...,...,...
3047,NaN,NaN,"Kyunggido, Kangwondo, Chungchongdo, Kyongsangd...",Korea (the Republic of)
3048,NaN,NaN,NaN,Lao People's Democratic Republic (the)
3049,NaN,NaN,North and Eastern regions,Sri Lanka
3050,NaN,NaN,"Kalutara district, Southwestern districts",Sri Lanka


In [45]:
# Filtrar DataFrame para detectar valores anómalos de latitud y longitud
df_eqanomalous = df_flood[
    df_flood['Latitude'].isin(anomalous_lat) | df_flood['Longitude'].isin(anomalous_lon)
]

# Crear pivot table para comparar las columnas de latitude, longitude, location, y country
df_eqanomalous_pivot = df_eqanomalous.pivot_table(
    index=['Location', 'Country'],
    values=['Latitude', 'Longitude'],
    aggfunc='first'
)

# Mostrar la pivot table
print(df_eqanomalous_pivot)


Empty DataFrame
Columns: []
Index: []


In [46]:
print(df_flood['Latitude'])
print(df_flood['Longitude'])

12      NaN
13      NaN
34      NaN
39      NaN
43      NaN
         ..
16620   NaN
16628   NaN
16629   NaN
16631   NaN
16634   NaN
Name: Latitude, Length: 5808, dtype: float64
12      NaN
13      NaN
34      NaN
39      NaN
43      NaN
         ..
16620   NaN
16628   NaN
16629   NaN
16631   NaN
16634   NaN
Name: Longitude, Length: 5808, dtype: float64


In [47]:
print(df_flood['Latitude'].dtype)
print(df_flood['Longitude'].dtype)

float64
float64


In [48]:
# # Examinar filas específicas para verificar si el proceso de limpieza ha manejado los valores anómalos correctamente.
# print(df_flood.loc[6000, 'Latitude'])
# print(df_flood.loc[6000, 'Longitude'])

In [49]:
# Valores nulos de Longitude y Latitude
df_flood[['Longitude', 'Latitude']].isnull().sum()

Longitude    4810
Latitude     4810
dtype: int64

## Valores Anómalos de Latitud y Longitud

In [50]:
len(anomalous_lat)

0

In [51]:
len(anomalous_lon)

0

#### Índices Anómalos de Latitud y longitud

In [52]:
anomalous_lat_idx = []
for index, lat in enumerate(anomalous_lat):
    if lat < -90 or lat > 90:
        anomalous_lat_idx.append(index)

print("Índices Anómalos de Latitud:", anomalous_lat_idx)

Índices Anómalos de Latitud: []


In [53]:
anomalous_lon_idx = []
for index, lon in enumerate(anomalous_lon):
    if lat < -90 or lat > 90:
        anomalous_lon_idx.append(index)

print("Índices Anómalos de Longitud:", anomalous_lon_idx)

Índices Anómalos de Longitud: []


## Outliers de Latitude y Longitude

In [54]:
# sns.boxplot(df_flood['Latitude'])

In [55]:
# sns.boxplot(df_flood['Longitude'])

## Valores faltantes

In [56]:
df_flood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5808 entries, 12 to 16634
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dis No             5808 non-null   object 
 1   Year               5808 non-null   int64  
 2   Seq                5808 non-null   int64  
 3   Disaster Subgroup  5808 non-null   object 
 4   Disaster Type      5808 non-null   object 
 5   Disaster Subtype   3564 non-null   object 
 6   Country            5808 non-null   object 
 7   ISO                5808 non-null   object 
 8   Region             5808 non-null   object 
 9   Continent          5808 non-null   object 
 10  Location           5512 non-null   object 
 11  Origin             3446 non-null   object 
 12  Associated Dis     1383 non-null   object 
 13  Dis Mag Value      5808 non-null   int64  
 14  Dis Mag Scale      5808 non-null   object 
 15  Latitude           998 non-null    float64
 16  Longitude          998

In [57]:
df_flood.isnull().sum()

Dis No                  0
Year                    0
Seq                     0
Disaster Subgroup       0
Disaster Type           0
Disaster Subtype     2244
Country                 0
ISO                     0
Region                  0
Continent               0
Location              296
Origin               2362
Associated Dis       4425
Dis Mag Value           0
Dis Mag Scale           0
Latitude             4810
Longitude            4810
Start Year              0
Start Month             0
Start Day               0
End Year                0
End Month               0
End Day                 0
Total Deaths            0
Total Affected          0
Total Damages Adj       0
dtype: int64

## Imputar valores anómalos con SimpleImputer

In [58]:
imputer = SimpleImputer(strategy='most_frequent')

# Seleccionar filas donde 'Longitude' está (isin) la lista de anomalous_lon
anomalous_lat_to_impute = df_flood['Latitude'].isin(anomalous_lat)

# Imputar los valores con la media a anomalous_lon en 'Longitude'
df_flood.loc[anomalous_lat_to_impute, 'Latitude'] = imputer.fit_transform(df_flood[['Latitude']])[anomalous_lat_to_impute]
df_flood['Latitude'].isnull().sum()

4810

In [59]:
imputer = SimpleImputer(strategy='most_frequent')

# Seleccionar filas donde 'Longitude' está (isin) la lista de anomalous_lon
anomalous_lon_to_impute = df_flood['Longitude'].isin(anomalous_lon)

# Imputar los valores anómalos con la media a anomalous_lon en 'Longitude'
df_flood.loc[anomalous_lon_to_impute, 'Longitude'] = imputer.fit_transform(df_flood[['Longitude']])[anomalous_lon_to_impute]
df_flood['Longitude'].isnull()

12       True
13       True
34       True
39       True
43       True
         ... 
16620    True
16628    True
16629    True
16631    True
16634    True
Name: Longitude, Length: 5808, dtype: bool

Imputar mediana a nulos con SimpleImputer a Longitude y Latitude

In [60]:
imputer = SimpleImputer(strategy='median')
# Seleccionar filas donde 'Longitude' sea null (NaN)
null_lon_rows = df_flood['Latitude'].isnull()
# Imputar los nulls en 'Longitude' estrategia mean
df_flood.loc[null_lon_rows, 'Latitude'] = imputer.fit_transform(df_flood[['Latitude']])[null_lon_rows]
df_flood['Latitude'].isnull().sum()

0

In [61]:
imputer = SimpleImputer(strategy='median')
# Seleccionar filas donde 'Longitude' sea null (NaN)
null_lon_rows = df_flood['Longitude'].isnull()
# Imputar los nulls en 'Longitude' estrategia mean
df_flood.loc[null_lon_rows, 'Longitude'] = imputer.fit_transform(df_flood[['Longitude']])[null_lon_rows]
df_flood['Longitude'].isnull().sum()

0

In [62]:
df_flood.head()

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
12,1906-0023-BEL,1906,23,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,36.71,1906,5,14,1906,5,14,6,0,0
13,1906-0024-BEL,1906,24,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,36.71,1906,4,0,1906,4,0,0,0,0
34,1911-0002-CHN,1911,2,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,...,36.71,1911,0,0,1911,0,0,100000,0,0
39,1915-0011-CHN,1915,11,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,...,36.71,1915,0,0,1915,0,0,0,3000000,0
43,1917-0013-CHN,1917,13,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,...,36.71,1917,0,0,1917,0,0,0,635000,0


# Pivot Tables

In [63]:
df_flood_origin_geolocation_pivot = df_flood.pivot_table(
    index='Origin',
    columns='Disaster Subtype',
    values=['Latitude', 'Longitude', 'Location'],
    aggfunc='sum',
    fill_value=0
)
df_flood_origin_geolocation_pivot

C:\Users\AleEng\AppData\Local\Temp\ipykernel_5580\3815334909.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df_flood_origin_geolocation_pivot = df_flood.pivot_table(


Latitude              \
Disaster Subtype                                 Coastal flood Flash flood   
Origin                                                                       
Abundant snowmelt and heavy rains                        0.000       0.000   
Active monsoonal rainfall                                0.000      16.705   
Bad weather associated with tropical storm Chris        16.705       0.000   
Break/release of dam/levy                                0.000      16.705   
Breif torrential rain                                    0.000      16.705   
...                                                        ...         ...   
West African Monsoon                                     0.000       0.000   
continuous rain                                          0.000       0.000   
heavy rain                                               0.000       0.000   
ice jam/break-up                                         0.000       0.000   
monsoonal rain                                           0.000       0.000   

                                                                    Longitude  \
Disaster Subtype                                 Riverine flood Coastal flood   
Origin                                                                          
Abundant snowmelt and heavy rains                        51.450          0.00   
Active monsoonal rainfall                                 0.000          0.00   
Bad weather associated with tropical storm Chris          0.000         36.71   
Break/release of dam/levy                                 0.000          0.00   
Breif torrential rain                                     0.000          0.00   
...                                                         ...           ...   
West African Monsoon                                     16.705          0.00   
continuous rain                                          27.720          0.00   
heavy rain                                               34.400          0.00   
ice jam/break-up                                         16.705          0.00   
monsoonal rain                                           16.705          0.00   

                                                                             
Disaster Subtype                                 Flash flood Riverine flood  
Origin                                                                       
Abundant snowmelt and heavy rains                       0.00          53.67  
Active monsoonal rainfall                              36.71           0.00  
Bad weather associated with tropical storm Chris        0.00           0.00  
Break/release of dam/levy                              36.71           0.00  
Breif torrential rain                                  36.71           0.00  
...                                                      ...            ...  
West African Monsoon                                    0.00          36.71  
continuous rain                                         0.00         104.07  
heavy rain                                              0.00          28.78  
ice jam/break-up                                        0.00          36.71  
monsoonal rain                                          0.00          36.71  

[340 rows x 6 columns]

In [64]:
df_flood_year_geolocation_pivot = df_flood.pivot_table(
    index='Year',
    columns='Disaster Subtype',
    values=['Latitude', 'Longitude', 'Location'],
    aggfunc='sum',
    fill_value=0
)
df_flood_year_geolocation_pivot

C:\Users\AleEng\AppData\Local\Temp\ipykernel_5580\2187748804.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df_flood_year_geolocation_pivot = df_flood.pivot_table(


Latitude                                Longitude  \
Disaster Subtype Coastal flood Flash flood Riverine flood Coastal flood   
Year                                                                      
1909                    16.705       0.000          0.000         36.71   
1911                     0.000       0.000         16.705          0.00   
1915                     0.000       0.000         16.705          0.00   
1917                     0.000       0.000         16.705          0.00   
1926                     0.000       0.000         16.705          0.00   
...                        ...         ...            ...           ...   
2019                     0.000     587.235        176.920          0.00   
2020                     0.000     611.115        217.175          0.00   
2021                     0.000     751.725        133.640          0.00   
2022                     0.000     451.035         33.410          0.00   
2023                     0.000      83.525         16.705          0.00   

                                             
Disaster Subtype Flash flood Riverine flood  
Year                                         
1909                    0.00           0.00  
1911                    0.00          36.71  
1915                    0.00          36.71  
1917                    0.00          36.71  
1926                    0.00          36.71  
...                      ...            ...  
2019                 1454.52         287.46  
2020                 1314.73         508.26  
2021                 1651.95         293.68  
2022                  991.17          73.42  
2023                  183.55          36.71  

[72 rows x 6 columns]

# Guardar dataset Flood limpio en csv

In [65]:
df.to_csv('../../Data/03Limpio/FloodLimpioCSV/03desastres_2floodcoordlimpias.csv')